In [1]:
!nvidia-smi

Sun Jul 16 16:18:03 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.40                 Driver Version: 536.40       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1660 Ti   WDDM  | 00000000:01:00.0  On |                  N/A |
| 45%   61C    P2             107W / 120W |   3232MiB /  6144MiB |      9%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

计算设备

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

查询可用gpu数量

In [3]:
torch.cuda.device_count()

1

定义了两个方便的函数， 这两个函数允许我们在不存在所需所有GPU的情况下运行代码。

In [5]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

查询张量所在的设备

In [6]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

存储在GPU上

In [7]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [8]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.4799, 0.2920, 0.7326],
        [0.6895, 0.1530, 0.4137]])

In [9]:
Z = X.cuda
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
<built-in method cuda of Tensor object at 0x000001D743AF5C60>


In [10]:
Y + Z

TypeError: unsupported operand type(s) for +: 'Tensor' and 'builtin_function_or_method'

神经网络与GPU

In [11]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [12]:
net(X)

tensor([[0.3884],
        [0.3884]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [13]:
net[0].weight.data.device

device(type='cuda', index=0)